In [1]:
import pandas as pd
import numpy as np

In [5]:
master = pd.read_pickle("master1.pkl")

In [6]:
len(master)

18033

In [7]:
master.describe()

,len,lineNo,blank,email,url,phone,sigdelimiter,special,words,name,...,posRatio,posRatioFromEnd,posRatioNB,lenRatio,lenRatioMax,less_avg_len,more_avg_len,less_avg_len75,less_avg_len50,entity_code
count,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,...,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000,18033.000000
mean,36.215050,45.928354,0.245217,0.024122,0.045694,0.103921,0.018854,0.007375,0.014917,0.014196,...,0.514307,0.485693,0.686805,0.970645,0.468741,0.524427,0.475573,0.457273,0.387734,0.083957
std,30.890788,55.882964,0.430228,0.153434,0.208826,0.305166,0.136014,0.085565,0.121225,0.118302,...,0.289338,0.289338,0.407083,0.861625,0.396866,0.499417,0.499417,0.498185,0.487247,0.277331
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002336,0.000000,0.003067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.263889,0.235849,0.344828,0.064516,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000
50%,32.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.512821,0.487179,0.666667,0.920000,0.410959,1.000000,0.000000,0.000000,0.000000,0.000000
75%,71.000000,58.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.764151,0.736111,1.000000,1.595238,0.911392,1.000000,1.000000,1.000000,1.000000,0.000000
max,255.000000,428.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,0.997664,2.875000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [61]:
master.iloc[:10,:10]

,line,filename,entity,len,lineNo,email,url,phone,sigdelimiter,special
0,"Lisa,",0.txt,no_entity,5,1,0,0,0,0,0
1,I apologize for the delay! This is a new one ...,0.txt,no_entity,239,2,0,0,0,0,0
2,Rhonda,0.txt,signature,6,3,0,0,0,0,0
3,"Hi Rhonda,",1.txt,no_entity,10,1,0,0,0,0,0
4,I know you are not the one who is working on ...,1.txt,no_entity,395,2,0,0,0,0,0
5,lisa,1.txt,signature,4,3,0,0,0,0,0
6,**********************************************...,1.txt,signature,70,4,0,0,0,0,1
7,This e-mail is the property of Enron Corp. and...,1.txt,signature,818,5,1,1,0,0,0
8,**********************************************...,1.txt,signature,70,6,0,0,0,0,1
9,"Leo/Monika,",10.txt,no_entity,11,1,0,0,0,0,0


In [62]:
master.columns

Index(['line', 'filename', 'entity', 'len', 'lineNo', 'email', 'url', 'phone',
       'sigdelimiter', 'special', 'words', 'name', 'endquote', 'tabs1',
       'tabs2', 'tabs3', 'punct20', 'punct50', 'punct90', 'reply',
       'startpunct', 'firstchar', 'replypunct', 'wrote', 'alphanum90',
       'alphanum50', 'alphanum10', 'title', 'nlines', 'len_avg', 'len_min',
       'len_max', 'nBlanks', 'nNonBlanks', 'nSig', 'posFromEnd', 'last',
       'prevlast', 'last5', 'last11', 'posRatio', 'posRatioNB', 'lenRatio',
       'less_avg_len', 'more_avg_len', 'less_avg_len75', 'less_avg_len50'],
      dtype='object')

# Convert to sklear-crfsuite format
each sentence should be converted to a list of dicts

In [63]:
import sklearn_crfsuite

In [64]:
def line2features(index, row, source):
    features = row_to_dict(row)

    # There is a previous one
    if row.lineNo > 1:
        try:
            prev_line = source.iloc[index - 1,]
        except IndexError as ex:
            print(index)
            print(row)
        features.update(row_to_dict(prev_line, "-1:"))
    else:
        features['BOS'] = True

    if (row.posFromEnd > 0) and (index < len(source.index) - 1):
        next_line = source.iloc[index + 1,]
        features.update(row_to_dict(next_line, "+1:"))
    else:
        features['EOS'] = True
    return features

In [65]:
def get_features(df, files):
    result = []
    for f in files:        
        curr = df[df.filename == f]
        result.append([line2features(index, row, curr) for index, (idx, row) in enumerate(curr.iterrows())])
    return result

In [66]:
def get_labels(df, files):
    result = []
    for f in files:        
        curr = df[df.filename == f]
        result.append([row.entity for index, (idx, row) in enumerate(curr.iterrows())])
    return result

In [67]:
def row_to_dict(row, prefix=""):
    features = {}
    for col in featured_columns:
        features[f"{prefix}{col}"] = row[col]
    return features

In [68]:
excluded_columns = ["line", "filename", "entity", "nSig", "firstchar", 'nlines',
 'len_avg',
 'len_min',
 'len_max',
 'nBlanks',
 'nNonBlanks',
 ]
featured_columns = [c for c in master.columns if c not in excluded_columns]

In [69]:
featured_columns

['len',
 'lineNo',
 'email',
 'url',
 'phone',
 'sigdelimiter',
 'special',
 'words',
 'name',
 'endquote',
 'tabs1',
 'tabs2',
 'tabs3',
 'punct20',
 'punct50',
 'punct90',
 'reply',
 'startpunct',
 'replypunct',
 'wrote',
 'alphanum90',
 'alphanum50',
 'alphanum10',
 'title',
 'posFromEnd',
 'last',
 'prevlast',
 'last5',
 'last11',
 'posRatio',
 'posRatioNB',
 'lenRatio',
 'less_avg_len',
 'more_avg_len',
 'less_avg_len75',
 'less_avg_len50']

In [70]:
row_to_dict(master.iloc[0,:],"-1:")

{'-1:len': 5,
 '-1:lineNo': 1,
 '-1:email': 0,
 '-1:url': 0,
 '-1:phone': 0,
 '-1:sigdelimiter': 0,
 '-1:special': 0,
 '-1:words': 0,
 '-1:name': 0,
 '-1:endquote': 0,
 '-1:tabs1': 0,
 '-1:tabs2': 0,
 '-1:tabs3': 0,
 '-1:punct20': 1,
 '-1:punct50': 0,
 '-1:punct90': 0,
 '-1:reply': 0,
 '-1:startpunct': 0,
 '-1:replypunct': 0,
 '-1:wrote': 1,
 '-1:alphanum90': 1,
 '-1:alphanum50': 0,
 '-1:alphanum10': 0,
 '-1:title': 1,
 '-1:posFromEnd': 2,
 '-1:last': 0,
 '-1:prevlast': 0,
 '-1:last5': 1,
 '-1:last11': 1,
 '-1:posRatio': 0.14285714285714285,
 '-1:posRatioNB': 0.3333333333333333,
 '-1:lenRatio': 0.058823529411764705,
 '-1:less_avg_len': 1,
 '-1:more_avg_len': 0,
 '-1:less_avg_len75': 1,
 '-1:less_avg_len50': 1}

In [71]:
filenames = master.filename.unique()

In [72]:
from sklearn.model_selection import train_test_split
train_filenames, test_filenames = train_test_split(filenames, test_size=0.2)

In [73]:
train = master[master.filename.isin(train_filenames)]
test = master[master.filename.isin(test_filenames)]

In [74]:
# all should be empty if all match
print([(x, y) for x, y in zip(sorted(train_filenames), sorted(train.filename.unique())) if x != y])
print([(x, y) for x, y in zip(sorted(test_filenames), sorted(test.filename.unique())) if x != y])
print([x for x in train_filenames if x in test_filenames])

[]
[]
[]


In [75]:
y_train = train.entity
y_test = test.entity

In [76]:
X_train = get_features(train, train_filenames)
y_train = get_labels(train, train_filenames)

In [77]:
X_test = get_features(test, test_filenames)
y_test = get_labels(test, test_filenames)

In [78]:
len(X_train)

216

In [79]:
len(y_train)

216

In [80]:
master[master.filename == 'gay-r_all_documents_383']

,line,filename,entity,len,lineNo,email,url,phone,sigdelimiter,special,...,prevlast,last5,last11,posRatio,posRatioNB,lenRatio,less_avg_len,more_avg_len,less_avg_len75,less_avg_len50


In [81]:
X_train[0]

[{'len': 257,
  'lineNo': 1,
  'email': 0,
  'url': 0,
  'phone': 1,
  'sigdelimiter': 0,
  'special': 0,
  'words': 0,
  'name': 0,
  'endquote': 0,
  'tabs1': 0,
  'tabs2': 0,
  'tabs3': 0,
  'punct20': 0,
  'punct50': 0,
  'punct90': 0,
  'reply': 0,
  'startpunct': 0,
  'replypunct': 0,
  'wrote': 1,
  'alphanum90': 1,
  'alphanum50': 0,
  'alphanum10': 0,
  'title': 0,
  'posFromEnd': 2,
  'last': 0,
  'prevlast': 0,
  'last5': 1,
  'last11': 1,
  'posRatio': 0.14285714285714285,
  'posRatioNB': 0.3333333333333333,
  'lenRatio': 2.763440860215054,
  'less_avg_len': 0,
  'more_avg_len': 1,
  'less_avg_len75': 0,
  'less_avg_len50': 0,
  'BOS': True,
  '+1:len': 7,
  '+1:lineNo': 2,
  '+1:email': 0,
  '+1:url': 0,
  '+1:phone': 0,
  '+1:sigdelimiter': 0,
  '+1:special': 0,
  '+1:words': 1,
  '+1:name': 0,
  '+1:endquote': 0,
  '+1:tabs1': 0,
  '+1:tabs2': 0,
  '+1:tabs3': 0,
  '+1:punct20': 0,
  '+1:punct50': 0,
  '+1:punct90': 0,
  '+1:reply': 0,
  '+1:startpunct': 0,
  '+1:replypu

In [82]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

In [83]:
crf.classes_

['no_entity', 'signature']

In [84]:
from sklearn_crfsuite import metrics

In [85]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=list(crf.classes_))

0.8724960481997445

In [34]:
crf.attributes_

['len',
 'lineNo',
 'blank',
 'email',
 'url',
 'phone',
 'sigdelimiter',
 'special',
 'words',
 'name',
 'endquote',
 'tabs1',
 'tabs2',
 'tabs3',
 'punct20',
 'punct50',
 'punct90',
 'reply',
 'startpunct',
 'replypunct',
 'wrote',
 'alphanum90',
 'alphanum50',
 'alphanum10',
 'title',
 'posFromEnd',
 'last',
 'prevlast',
 'last5',
 'last11',
 'posRatio',
 'posRatioFromEnd',
 'posRatioNB',
 'lenRatio',
 'lenRatioMax',
 'less_avg_len',
 'more_avg_len',
 'less_avg_len75',
 'less_avg_len50',
 'entity_code',
 'BOS',
 'EOS',
 '+1:len',
 '+1:lineNo',
 '+1:blank',
 '+1:email',
 '+1:url',
 '+1:phone',
 '+1:sigdelimiter',
 '+1:special',
 '+1:words',
 '+1:name',
 '+1:endquote',
 '+1:tabs1',
 '+1:tabs2',
 '+1:tabs3',
 '+1:punct20',
 '+1:punct50',
 '+1:punct90',
 '+1:reply',
 '+1:startpunct',
 '+1:replypunct',
 '+1:wrote',
 '+1:alphanum90',
 '+1:alphanum50',
 '+1:alphanum10',
 '+1:title',
 '+1:posFromEnd',
 '+1:last',
 '+1:prevlast',
 '+1:last5',
 '+1:last11',
 '+1:posRatio',
 '+1:posRatioFromEn

# Grid Search

In [86]:
import scipy
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics

In [36]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=50,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average='weighted', labels=['no_entity', 'signature'])

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

G:\Programs\Anaconda2\envs\thesis-enron\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 52.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=CRF(algorithm='lbfgs', all_possible_states=None,
                                 all_possible_transitions=True, averaging=None,
                                 c=None, c1=None, c2=None,
                                 calibration_candidates=None,
                                 calibration_eta=None,
                                 calibration_max_trials=None,
                                 calibration_rate=None,
                                 calibration_samples=None, delta=None,
                                 epsilon=None, error_sensitive=None, gamma=None,
                                 keep_...
                   iid='deprecated', n_iter=50, n_jobs=-1,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001DBA3D4BAC8>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001DBA3D4B988>},
        

In [37]:
best = rs.best_estimator_

In [38]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

best params: {'c1': 0.6031404279524103, 'c2': 0.03969051576126347}
best CV score: 0.9999224679901436
model size: 0.02M


In [148]:
best.attributes_

['len',
 'lineNo',
 'email',
 'url',
 'phone',
 'sigdelimiter',
 'special',
 'words',
 'tabs1',
 'tabs2',
 'punct20',
 'punct50',
 'punct90',
 'reply',
 'startpunct',
 'wrote',
 'alphanum90',
 'alphanum50',
 'alphanum10',
 'title',
 'nlines',
 'len_avg',
 'len_min',
 'len_max',
 'nBlanks',
 'nNonBlanks',
 'nSig',
 'posFromEnd',
 'last',
 'prevlast',
 'last5',
 'last11',
 'posRatio',
 'posRatioNB',
 'lenRatio',
 'less_avg_len',
 'more_avg_len',
 'less_avg_len75',
 'less_avg_len50',
 'BOS',
 '+1:len',
 '+1:lineNo',
 '+1:email',
 '+1:url',
 '+1:phone',
 '+1:sigdelimiter',
 '+1:special',
 '+1:words',
 '+1:name',
 '+1:tabs1',
 '+1:punct20',
 '+1:punct50',
 '+1:punct90',
 '+1:reply',
 '+1:startpunct',
 '+1:replypunct',
 '+1:wrote',
 '+1:alphanum90',
 '+1:alphanum50',
 '+1:alphanum10',
 '+1:title',
 '+1:nlines',
 '+1:len_avg',
 '+1:len_min',
 '+1:len_max',
 '+1:nBlanks',
 '+1:nNonBlanks',
 '+1:nSig',
 '+1:posFromEnd',
 '+1:last',
 '+1:prevlast',
 '+1:last5',
 '+1:last11',
 '+1:posRatio',
 '+1

# SVM
 take as input two arrays: an array X of shape (n_samples, n_features)

In [87]:
X_s

,line,filename,entity,len,lineNo,email,url,phone,sigdelimiter,special,...,prevlast,last5,last11,posRatio,posRatioNB,lenRatio,less_avg_len,more_avg_len,less_avg_len75,less_avg_len50
0,"Lisa,",0.txt,no_entity,5,1,0,0,0,0,0,...,0,1,1,0.142857,0.333333,0.058824,1,0,1,1
1,I apologize for the delay! This is a new one ...,0.txt,no_entity,239,2,0,0,0,0,0,...,1,1,1,0.285714,0.666667,2.811765,0,1,0,0
2,Rhonda,0.txt,signature,6,3,0,0,0,0,0,...,0,1,1,0.428571,1.000000,0.070588,1,0,1,1
3,"Hi Rhonda,",1.txt,no_entity,10,1,0,0,0,0,0,...,0,0,1,0.090909,0.166667,0.043103,1,0,1,1
4,I know you are not the one who is working on ...,1.txt,no_entity,395,2,0,0,0,0,0,...,0,1,1,0.181818,0.333333,1.702586,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3200,\t(1)\tDetermination of the Canadian entity na...,99.txt,no_entity,71,4,0,0,0,0,0,...,0,1,1,0.307692,0.500000,0.537879,1,0,1,0
3201,\t(2)\tObtaining financial information for thi...,99.txt,no_entity,103,5,0,0,0,0,0,...,0,1,1,0.384615,0.625000,0.780303,1,0,0,0
3202,\t(3)\t3 - 5 business days for the pipeline to...,99.txt,no_entity,64,6,0,0,0,0,0,...,0,1,1,0.461538,0.750000,0.484848,1,0,1,1
3203,The bottom line is that unless items 1 & 2 can...,99.txt,no_entity,575,7,0,0,0,0,0,...,1,1,1,0.538462,0.875000,4.356061,0,1,0,0
